# Project tasks

- Please rename this file so that you know which copy you have been working in. Keep a copy safe (especially if you are working in the online Jupyter service). You can download a copy by choosing -File- then -Download as- Notebook from the menu above. 
- Complete all of the tasks 
- Make sure to use good code style, and comment your code appropriately.

---

## Task 1 - Code review

This task is to write a code review, *not* to write python code to solve the problem brief. 

A colleague has been asked to write some code to answer the following brief:

---

### Brief:
The determinant of an $n\times n$ matrix $A$ can be calculated recursively by "row/column expansion", for any given column $j$:

$$
\textrm{det}(A) = \sum_{i=1}^n (−1)^{(i+j)}\,\,a_{ij}\,\, \textrm{det}(\bar{A}_{ij})
$$

where $a_{ij}$ is the element of $A$ in the $i$-th row and $j$-th column, and $\bar{A}_{ij}$ is the $(n − 1)\times (n − 1)$ matrix obtained from $A$ by deleting the $i$-th row and $j$-th column.

The above formula works for any $j$ (you can just use $j = 1$: expansion of the first column).

You should:

* Write a function to (recursively) work out the determinant of a two-dimensional Numpy array without using the inbuilt functions. 

* Write a further function which determines the volume of the parallelepiped spanned by the vectors $(x_1,y_1,z_1)$, $(x_2,y_2,z_2)$, $(x_3,y_3,z_3)$, which is given by the absolute value of:

$$
\textrm{det}\left(\begin{array}{ccc}
x_1 & y_1 & z_1\\
x_2 & y_2 & z_2\\
x_3 & y_3 & z_3
\end{array}\right)
$$
using your determinant function.


---

### Your task:

You have been asked to write a review of their code. Here is the code they produced:

In [ ]:
from numpy import *

def my_det(A,n):
# Calculate the determinant of a matrix
    if n==1:
        return A[0,0]  #Return a single value in this case
    else:
     det = 0.0
     for i in range(0,n-1):
          # Remove the row and column:  
          B = delete(A,i,axis=0)
          C = delete(B,0,axis=1)
          #print(C)
          det = det + (-1)**(i+1)*A[i,0]*my_det(C,n-1) #implement the formula
    return det

def paraVol(a,b,c):
  '''
  Get the volume of a parallelepiped spanned by a,b,c.
  '''
  A = array([[a[0],a[1],a[2]],[b[0],b[1],b[2]],[c[0],c[1],c[2]]])
  return my_det(A,3)

 # Test on the identity matrix:
A = identity(3)
print(my_det(A,3))
print(linalg.det(A))  #compare with built-in function

You should write your review here. 
Things you could choose to discuss:
- Code structure 
- Code style 
- Does it answer the brief?
- Does it work? If not could it be fixed?
- Can you explain what it does?

Keep your answer relatively brief (approx. 500 words).

# Code Review

In my_det(), the 'n' should be calculated inside the function, as it is dependent entirely on 'A'. Its calculation would be simple, using the 𝚕𝚎𝚗𝚐𝚝𝚑() function on the first element of the input array. It should be put in the docstring for my_det() that A should be square, and there could also be error handling for a non-square input. Additionally, the base case should be for n=2, as it is simply (ad-bc). Changing the base case to this reduces the number of function calls, and the number of elementary operations - improving the efficiency. Still with my_det(), $\texttt{from numpy import *}$ is advised against in Python. When examining the code in future, it is harder to discern which functions and methods are from numpy. Furthermore, if a user wishes to add to the script, they must be careful not to overwrite any numpy objects with their own. 

Furthermore, the user has failed to create a docstring in my_det(), as docstrings are created with triple quotes. Even supposing it worked, the ‘docstring’ is not descriptive. For example it fails to mention what type and dimensions arguments should be. Similarly, the docstring for paraVol() is poor, for the same reason. Following on from the theme of comments, "#implement the formula" is pointless, as it describes nothing. Something like "Calculate the determinant of A recursively using expansion of the first column" would provide more detail to readers. "#print(C)" is poorly styled, and is a debugging technique that should not have been left in.

The function names are inconsistent which is poor style. PEP8 recommends lower case words separated by underscores, but the user should focus on keeping the names consistent. 


The style for paraVol() is poor - the matrix construction should be broken up onto multiple lines.

The brief says "without using the inbuilt functions”, but the code for my_det() uses the numpy function delete(). Additionally, the code for both functions does not work.

Consider my_det(), and the following matrix:

$$
\left(\begin{array}{cc} 
1 & 2\\
2 & 4
\end{array}\right)
$$ 


The rows are linearly dependent, meaning a 0 determinant, but applying my_det() to this matrix returns a determinant of -4. The way to fix the function is by adding 1 to the upper end of the loop range, and adding 1 to the exponent of -1 in the updating of 'det'. We need the loop to occur n times, as the formula states. The exponents are a mathematical tool, so their values should relate to indexing starting at 1 (as we do in mathematics when considering matrices), instead of 0.

Regarding paraVol(), negative volumes are possible, as determinants can be negative. This can be fixed with the abs() function, assuming my_det() works correctly.It also accepts arguments from higher dimensional space than 3D, which would not be a parallelepiped.  

paraVol() combines the the first 3 elements of each argument array into a matrix, and calculates the determinant of that matrix to return the volume.

my_det() recurisvely calculates the determinant, with the base case being n=1  (A is mathematically scalar). If A is non-scalar, the code deletes row index 'i' and column 1 (index j), and uses the given formula to get the determinant of the reduced matrix in the same way until the base case. Once the determinant of the reduced matrix has been calculated, the determinant of the current matrix is updated as per the formula. This is repeated for all rows.

---

## Task 2 - K-Nearest-neighbours

### Task 2a - First implementation

For this task you will need to implement a [K-nearest-neighbours](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) algorithm for classification from scratch (**important - do not use a version of this algorithm from another module e.g SKLearn - you need to write the functions as directed yourself**). The K-nearest-neighbours algorithm is a classic machine learning algorithm used for [classification problems](https://en.wikipedia.org/wiki/Statistical_classification). Classifying a data item from a given dataset means deciding which of a number of classes the item belongs to. This is done using a *training set*, containing a number of data items with known class.

We will consider first a simple implementation of the 3-nearest-neighbour version of the algorithm. The algorithm proceeds as follows.

For each item in the dataset, we find the 3 nearest items (and their respective classes) in the training set. We then attribute a class to our data item by majority voting amongst the classes of the 3 nearest neighbours. In the case where three classes are tied in the vote, we resolve the tie by choosing the class of whichever neighbour is the nearest in the training set. (In practice this might not be a very good thing to do - but we are just constructing a simple implementation for now). Finally, we need to output the class we have decided on for each item in our original dataset.

The data is contained in two files `main_data.txt` and `train_data.txt`.
* Each item in `main_data.txt` must be classified into one of 7 classes, using the training set `train_data.txt`.
* Each line of `train_data.txt` defines 1 training item, as a vector of floating-point values, followed by its class label (a single integer in the range 1-7).
* Each line of `main_data.txt` defines 1 data item, as only a vector of floating-point values, without a class label.

To do this write 3 functions:

- **`data_setup()`** which reads in both the dataset contained in `main_data.txt`, and the training dataset contained in `train_data.txt` into two lists - returning the two lists (`main_data`, and `train_data`) from the function. The lists containing each item of the dataset should use appropriate types for each element of the list. 
- **`dist_vect(item,train_data)`** which takes as arguments one element of the list `main_data`, and the list `train_data`, both from the output of the data_setup function. The function should calculate the Euclidean distance from item in the dataset to each item in the training dataset in order. The function should return a list of (distance, class) tuples.
- **`decide_class(dv)`** which takes the output list of tuples of the dist_vect function as an argument, and uses it to return the class of the item in the dataset.

- Lastly use your functions to obtain a list containing the calculated class for each of the element in the `main_data` list you have constructed.

# Writing 3 Functions

In [ ]:
def data_setup():
    '''
    data_setup()
    
    A function that reads in the training and testing data from .txt files which
    have the form of a csv. The names of the files are 'train_data.txt' and
    'main_data.txt'. These are fixed, and as such the function takes no arguments. 
    
    
    To assign, use tuple unpacking. 
    
    Returns
    -------------
    
    (main_data, train_data) : Tuple. 
        Each element (data) is a list of tuples. Main_data tuples have length n, 
        and train_data tuples have length n+1, due to including the label (aka class). 
    
    '''
    train_data = list()
    main_data = list()
    

    
    # As mentioned, each item is an observation, with various features and a class as the final column
    #
    # For each item in training data, split the item string read in by open() on commas to, 
    # get a list of numeric strings (e.g. '3.14') and convert each element of the resulting 
    # list into a float. 
    #
    # Then convert the last element of the item (i.e. the class) into an integer
    #
    # Convert the list to a tuple for speed purposes in future use, and append this tuple to train_data
    
    with open('train_data.txt', 'r') as traindata:
        for line in traindata:
            item_list_train = [float(num) for num in line.split(',')]
            item_list_train[-1] = int(item_list_train[-1])
            train_data.append(tuple(item_list_train))
    traindata.close()
    
    # As above, without the conversion step, as main_data does not have known classes
    
    
    with open('main_data.txt', 'r') as maindata:
        for line in maindata:
            item_list_main = [float(num) for num in line.split(',')]
            main_data.append(tuple(item_list_main))
    maindata.close()
    
    return (main_data, train_data)

def dist_vect(item, train_data):
    '''
    dist_vect(item, train_data)
    
    Calculates the L2 distance between the item, and each item (tuple) in train_data
    and returns a list of tuples with this distance, and the class of the training item.
    
    
    Arguments
    ------------
    item: tuple of length n
    train_data: List of tuples, each tuple with length n+1 
    
    Returns
    -------------
    distclasslist: A list of (distance, class) tuples
        The L2 distance between the item, and each tuple in train_data, and 
        the class corresponding to the tuple in train_data.
    
    Examples
    ---------
    
    Data for input to function
    
    >>> item_ex =       (0, 5, 2, 1, 2, 7, 8)
    >>> train_data_ex = [(2, 2, 5, 1, 2, 5, 7, 1), (6, 2, 5, 1, 1, 3, 5, 2), (3, 3, 1, 1, 7, 3, 2, 2)]
    
    Output          
    
    >>> print(dist_vect(item = item_ex, train_data = train_data_ex))
    [(5.196152422706632, 1), (8.94427190999916, 2), (9.539392014169456, 2)]
    
    (5.1962, 1) means the L2 distance between item_ex and the first item of
    the training data is 5.1962, and the class of the first item in the
    training data is 1. 
    
    '''
    
    # Will store (distance, class) tuples here
    dist_class_list = list()
    
    # Only want to iterate for each feature in the item (which is unclassified)  
    main_length = len(item)

 
    # Create list of squared differences, sum this list, and square root the sum, to
    # calculate Euclidean distance.
    
    # Then extract the class of the training item, which is the final element in train_item
    
    for train_item in train_data:
        dist = sum([ (item[i] - train_item[i]) ** 2 for i in range(main_length) ]) ** 0.5
        train_class = train_item[-1]
        dist_class_list.append((dist, train_class))
        
    return dist_class_list


def decide_class(dv):
    '''
    decide_class(dv)
    
    Makes a prediction for the class of the item that created dv, by using a nearest-neighbours
    decision methodology, with k = 3. 
    
    
    Arguments
    -------------
    dv: A list of (distance, class) tuples
        Of the form outputted by the dist_vect functions. 
    
    
    Returns
    -------------
    nn_class: Integer
        The predicted class of the item that produced the distance vector dv. This
        predicted class is calculated using k-nearest-neighbours classification
        where k = 3, and using simple majority voting. 
        
    
    Examples
    -------------
    
    Using distance vector output example from dist_vect() docstring
    
    >>> dv_ex = [(5.196152422706632, 1), (8.94427190999916, 2), (9.539392014169456, 2)]
    >>> print(decide_class(dv_ex))
    2
    
    Modifying slightly so there is no modal class
    
    >>> dv_ex_2  =  [(5.196152422706632, 1), (8.94427190999916, 2), (9.539392014169456, 3)]
    >>> print(decide_class(dv_ex_2))
    1
    '''
    
    
    # Find 3 nearest. Sorted automatically sorts by first element
    
    dv_nearest_k = sorted(dv)[0:3]
    
    # Count classes generally
    # find highest integer class in dv
    # Create countlist, where indices correspond to class 
    # i.e. index 1 contains counts of class 1 etc. up to the highest class
    # count_list will automatically expand if more classes are added
    
    highest_class = max(dv, key = lambda x: x[1])[1]
    count_list = [0]*(highest_class+1)
    
    # Extract class from tuple, and add 1 to corresponding index in countlist to keep track of count
    # Recalling that class corresponds to list index. 
    for dv_tuple in dv_nearest_k:
        dv_class = dv_tuple[1]
        count_list[dv_class] += 1
    
    # classandcount matches each index (class) with its count, tuple of (class, count) tuples
    
    class_and_count = tuple(enumerate(count_list))
    
    # If one class greatest, return it
    # First, find maximum count in countlist

    max_count = max(count_list)
    
    # Create list of the indices (classes) where their count is the max count

    max_list = [index for index, count in class_and_count if count == max_count]
    
    # DECISION
    
    # If max_list has length 1, there is a unique maximum, hence return that class
    # Otherwise find the nearest neighbour (minimum distance class)
    
    
    if len(max_list) == 1:
        nn_class = max_list[0]
    else:
        # dv_nearest_k already sorted
        # First element is a (distance, class) tuple, use [1] to access class
        nn_class = dv_nearest_k[0][1]
    
    return nn_class

## Lastly use your functions to obtain a list containing the calculated class

In [ ]:
# Create function to predict the classes. 

def predict_knn_classes(datasets, distancecalc, classdecision):
    '''
    predict_knn_classes(datasets, distancecalc, classdecision)
        Function that outputs the predicted classes of the main_data, given a distance calculation and
        a particular decision methodology with the number of nearest neighbours considered set to 3 by
        default.

    Arguments
    -------------
    datasets: Function
        Outputs a (main_data, train_data) tuple with each element being a list of tuples
    
    distancecalc: Function
        Calculates the distance from a tuple from the main_data list (tuples have length n) to each tuple 
        in the train_data (tuples have length n+1).
        
        Outputs a list of (distance, class) tuples, where the class is from the last element of the tuple 
        in the train_data.
    
    classdecision: Function
        Uses the output from distancecalc to classify the item passed into distanecalc, given a k-nearest-neighbours 
        method with majority voting, using k=3. 

    Returns
    -------------
    main_class_predictions: List of integers
        A list of class predictions for each item in main_data. 

    '''
    # Read and assign data using function passed through to datasets
    # Initialise a list of predictions
    
    main_data, train_data = datasets()
    main_class_predictions = list()
    
    # For each unclassified item, create the list of (distance, class) tuples using the 
    # function passed to distancecalc
    
    # Using this list, predict the most appropriate class using the decision methodology
    # provided in the function passed to classdecision, and append it to the list of predictions
        
    for item in main_data:
        dv = distancecalc(item, train_data = train_data)
        main_class_predictions.append(classdecision(dv = dv))
    
    
    return(main_class_predictions)

# Use the 3 functions requested in the task as inputs to my function to
# predict the class of each item in the main_data.txt. 

print('L2 norm predictions: \n')
print(predict_knn_classes(data_setup, dist_vect, decide_class)) 

### Task 2b - Changing the distance measure

Next write two different replacements for the **dist_vect(item,train_data)** function. The function you have written above calculates the distance between data items using Euclidean distance as:
$$d(x,y) = \left(\sum_{i=1}^n (x_i-y_i)^2\right)^{1/2}$$

For this task write two more versions of the distance function using different distance metrics.

- Firstly **dist_vectL1(item,train_data)** where distance is determined by using:

$$d(x,y) = \sum_{i=1}^n \left|x_i-y_i\right|$$

- Next, **dist_vectLinf(item,train_data)** where distance is determined by using:

$$d(x,y) = \max\limits_{i=1..n}\left|x_i-y_i\right|$$

These are the $L_1$ and $L_\infty$ norms respectively. 

- Lastly use the new functions to obtain a list containing the calculated class for each of the element in the main_data list you have constructed.

In [ ]:
def dist_vectL1(item, train_data):
    '''
    dist_vectL1(item, train_data)
        Calculates the L1 distance between the item, and each item (tuple) in train_data and returns a list of 
        tuples with this distance, and the class of the training item.
    
    Arguments
    ------------
    item: tuple of length n
    train_data: List of tuples, each tuple with length n+1 
    
    Returns
    -------------
    distclasslist: A list of (distance, class) tuples
        The L1 distance between the item, and each tuple in train_data, and 
        the class corresponding to the tuple in train_data.
    
    Examples
    -------------
    
    Data for input to function
    
    >>> item_ex =       (0, 5, 2, 1, 2, 7, 8)
    >>> train_data_ex = [(2, 2, 5, 1, 2, 5, 7, 1), (6, 2, 5, 1, 1, 3, 5, 2), (3, 3, 1, 1, 7, 3, 2, 2)]
    
    Output          
    
    >>> print(dist_vectL1(item = item_ex, train_data = train_data_ex))
    [(11, 1), (20, 2), (21, 2)]
    
    (11, 1) means the L1 distance between item_ex and the first item of
    the training data is 11, and the class of the first item in the
    training data is 1.
    
    '''
    
    # Initialise list of (distance, class) tuples
    dist_class_list = list()
    
    # Only want to iterate for each feature in the unclassified item
    main_length = len(item)

    # Create a list of the absolute differences between each feature of the item and the training item
    # and then sum this, and append it to dist_class_list, along with the class of the training item. 
    
    for train_item in train_data:
        dist = sum([ abs((item[i] - train_item[i])) for i in range(main_length) ])
        train_class = train_item[-1]
        dist_class_list.append((dist, train_class))
        
    return dist_class_list

def dist_vectLinf(item, train_data):
    '''
    dist_vectLinf(item, train_data) 
        Calculates the L-infinity distance between the item, and each item (tuple) in train_data and returns 
        a list of tuples with this distance, and the class of the training item. 
    
    Arguments
    ------------
    item: tuple of length n
    train_data: List of tuples, each tuple with length n+1 
    
    Returns
    -------------
    distclasslist: A list of (distance, class) tuples
        The L-infinity distance between the item, and each tuple in train_data, and 
        the class corresponding to the tuple in train_data.
    
    Examples
    ---------
    
    Data for input to function
    
    >>> item_ex =       (0, 5, 2, 1, 2, 7, 8)
    >>> train_data_ex = [(2, 2, 5, 1, 2, 5, 7, 1), (6, 2, 5, 1, 1, 3, 5, 2), (3, 3, 1, 1, 7, 3, 2, 2)]
    
    Output          
    
    >>> print(dist_vectLinf(item = item_ex, train_data = train_data_ex))
    [(3, 1), (6, 2), (6, 2)]
    
    (3, 1) means the LInf distance between item_ex and the first item of
    the training data is 5.1962, and the class of the first item in the
    training data is 1.
    
    '''
    # Initialise list of (distance, class) tuples
    dist_class_list = list()
    
    # Only want to iterate for each feature in the unclassified item
    main_length = len(item)

    # Create a list of the absolute differences between each feature of the item and the training item
    # and then find the maximum difference, and append it to dist_class_list, along with the class of 
    # the training item. 
    
    for train_item in train_data:
        dist = max([ abs((item[i] - train_item[i])) for i in range(main_length) ])
        train_class = train_item[-1]
        dist_class_list.append((dist, train_class))
    
    return dist_class_list



print('L1 Norm predictions \n')
print(predict_knn_classes(datasets=data_setup, distancecalc=dist_vectL1,classdecision=decide_class), '\n')
print('L infinity Norm predictions \n')
print(predict_knn_classes(datasets=data_setup, distancecalc=dist_vectLinf,classdecision=decide_class))




### Task 2c - Changing the decision methodology

The simple majority-voting implemented in your `decide_class()` function is unweighted --- once the 3 nearest neighbours are found, they each have 1 vote, regardless of which is closest (unless there is a tie). An alternative is *weighted voting*, where each of the $k$ nearest neighbours is attributed a coefficient, such that the closest neighbours carry more weight in the result.

Write a function `decide_class_wk(dv,k)` which also takes the number $k$ of nearest neighbours as an input argument. Your function should attribute a weight $w_j$ to each of the $k$ nearest neighbours, inversely proportional to their distance from the data item:

$$
w_j = \frac{1}{d_j(x,y)}\, , \quad j \in \{1,\ldots,k\}
$$

The score for each class is determined by the sum of the weights of each item in that class amongst the $k$ nearest neighbours. The class with the highest score is chosen for the data item.

(Note that the simple majority-voting corresponds to setting all the weights to 1.)

Test your function on the dataset, for $k=3, 8,$ and $25$.

## Define decide_class_wk(dv, k)

In [ ]:
def decide_class_wk(dv, k):
    '''
    decide_class_wk(dv, k)
        Calculates the most appropriate class for an item which produced a list of (distance, class) tuples 
        against a training set, using k-nearest-neighbours classification.
    
    Arguments
    -------------
    dv: A list of (distance, class) tuples
        Of the form outputted by the dist_vect functions. 
    k:  Integer
        Number of nearest neighbours to consider before applying the decision
        methodology
    
    Returns
    -------------
    nn_class: Integer
        The predicted class of the item that produced the distance vector dv. This
        predicted class is calculated using k-nearest-neighbours classification
        with weighted scoring.
    
    Examples
    -------------
    
    >>> dv_ex = [(5.196152422706632, 1), (8.94427190999916, 2), (9.539392014169456, 2)]
    >>> print(decide_class_wk(dv_ex, 3))
    2
    
    Modifying so effect of weights can be seen.
    
    With weight 1, class 2 would win by majority. But with weighted voting, the fact class 1 is very 
    close to the item outweights the fact class 2 has more points near the item. 
    
    >>> dv_ex_2  =  [(1, 1), (8, 2), (9, 2)]
    >>> print(decide_class_wk(dv_ex_2, 3))
    1

    '''
    # Get nearest k
    dv_nearest_k = sorted(dv)[0:k]
    
    # Find highest class in dv
    # Initialise score_list, which will store score, where index of score_list 
    # corresponds to class. 
    # score_list will automatically expand if more classes are added
    
    highest_class = max(dv, key = lambda x: x[1])[1]
    score_list = [0]*(highest_class+1)
    
    # Calculate weight and extract class
    # Update the total weight corresponding to the class in scorelist.
    
    for dv_tuple in dv_nearest_k:
        weight = (dv_tuple[0]) ** (-1)  
        dv_class = dv_tuple[1]
        score_list[dv_class] += weight
    
    # Find index (hence class) corresponding to highest score, and return it.
    
    nn_class = score_list.index(max(score_list))
    return nn_class


## Test function for k $\in$ {3, 8, 25}

In [ ]:
# Create function to predict the classes. 

def predict_knn_classes(datasets, distancecalc, classdecision, k):
    '''
    predict_knn_classes(datasets, distancecalc, classdecision, k)
        Function that outputs the predicted classes of the main_data, given a distance calculation and
        a particular decision methodology with the number of nearest neighbours considered. 

    Arguments
    -------------
    datasets: Function
        Outputs a (main_data, train_data) tuple with each element being a list 
        of tuples
    k: Integer
        How many nearest neighbours to consider when applying decision methodology
    
    distancecalc: Function
        Calculates the distance from a tuple from the main_data list (tuples 
        have length n) to each tuple in the train_data (tuples have length n+1).
        Outputs a list of (distance, class) tuples, where the class is from the
        last element of the tuple in the train_data.
    
    classdecision: Function
        Uses the output from distancecalc to classify the item passed into
        distanecalc, given a k-nearest-neighbours method with majority voting,
        using k=3. 

    Returns
    -------------
    main_class_predictions: List of integers
        A list of class predictions for each item in main_data. 

    '''
    # Read and assign data using function passed through to datasets
    # Initialise a list of predictions

    # Make a prediction for each item in the main_data using the distance calculation
    # given, and the class decision method. 
    
    main_data, train_data = datasets()
    main_class_predictions = list()
    
    # For each unclassified item, create the list of (distance, class) tuples using the 
    # function passed to distancecalc
    
    # Using this list, predict the most appropriate class using the decision methodology
    # provided in the function passed to classdecision, and append it to the list of predictions
        
    for item in main_data:
        dv = distancecalc(item, train_data = train_data)
        main_class_predictions.append(classdecision(dv = dv, k = k))
    
    
    return(main_class_predictions)

# Use the 3 functions requested in the task as inputs to my function to
# predict the class of each item in the main_data.txt. 

for i in (3, 8, 25):
    print('For k =', i, 'using L2 distance, the predictions for the main_data are: \n')
    print(predict_knn_classes(data_setup, dist_vect, decide_class_wk, k = i), '\n')
